### IMPORT ALL COMMON PACKAGES AND FUNCTIONS

In [2]:
# from ipynb.fs.full.common import *
%run common.ipynb

### DEFINE GLOBAL VARIABLES 

In [3]:
ACTIVE_SHAM = 'Sham'
GROUP1 = 'PRE-AVG'
GROUP2= 'POST-AVG'
COMPUTE_DATA = True
mapping = {0: 'theta', 1: 'alpha', 2:'beta'}
FREQ_BANDS = {
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
}
folder1 = '2Pre-TDCS'
folder2 = '4Post-TDCS'
connectivity = {}
binarized_matrix = {}


Group 1 data

In [4]:
if COMPUTE_DATA:
    c = 0
    PSD_1 = []
    con_mat_1 = {'theta': [], 'alpha': [], 'beta': []}
    for ELECTRODES in ['32electrodes', '32electrodes-old']:
        folder_path = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', ELECTRODES,  ACTIVE_SHAM, folder1)
        for patient in os.listdir(folder_path):
            print(patient)
            file = glob.glob(os.path.join(folder_path, patient) + '/*' + 'Close.easy') if ELECTRODES == '32electrodes' else glob.glob(os.path.join(folder_path, patient) + '/*' + 'EEG.easy')
            for _ in file:
                raw_1 = data_transformation_easy(_)
                raw_1 = g1_preprocess(raw_1)
                psd_1, frequencies = mne.time_frequency.psd_array_welch(raw_1.get_data(), fmin=FMIN, fmax=FMAX, sfreq=SFREQ, n_fft = 8192)
                PSD_1.append(psd_1)

                if c==0:
                    duration = 4.0
                    overlap = 2.0 

                    samples_per_epoch = int(duration * SFREQ)
                    samples_per_overlap = int(overlap * SFREQ)

                    start, stop = 0, samples_per_epoch
                    events = []
                    while stop <= len(raw_1):
                        events.append([start, 0, 1]) 
                        start += samples_per_overlap
                        stop += samples_per_overlap

                    events = np.array(events)

                epochs_1 = mne.Epochs(raw_1, events, tmin=0, tmax=duration, baseline=(0,0), detrend=1,
                            picks=None, preload=True, reject=None)
                n_channels = len(raw_1.info['ch_names'])
                for i in mapping:
                    con = connectivity_matrix(epochs_1, i)
                    con_mat_1[mapping[i]].append(con.get_data().reshape((n_channels, n_channels)))
                c +=1 

    print(con_mat_1)

Kuldeep
Creating RawArray with float64 data, n_channels=32, n_times=74999
    Range : 0 ... 74998 =      0.000 ...   149.996 secs
Ready.
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hann window with 0.0546 passband ripple and 44 dB stopband attenuation
- Lower transition bandwidth: 0.01 Hz
- Upper transition bandwidth: 11.25 Hz
- Filter length: 155001 samples (310.002 s)



/tmp/ipykernel_19086/607340406.py:2: RuntimeWarning: filter_length (155001) is longer than the signal (74999), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(method= 'fir',
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Effective window size : 16.384 (s)
Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computin

/tmp/ipykernel_19086/607340406.py:2: RuntimeWarning: filter_length (155001) is longer than the signal (74999), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(method= 'fir',
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Effective window size : 16.384 (s)
Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computin

...

Effective window size : 16.384 (s)
Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computin

/tmp/ipykernel_19086/607340406.py:2: RuntimeWarning: filter_length (155001) is longer than the signal (74999), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(method= 'fir',
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Effective window size : 16.384 (s)
Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computin

/tmp/ipykernel_19086/607340406.py:2: RuntimeWarning: filter_length (155001) is longer than the signal (74999), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(method= 'fir',
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Effective window size : 16.384 (s)
Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computin

Group 2 data

In [5]:
if COMPUTE_DATA:
    c = 0
    con_mat_2 = {'theta': [], 'alpha': [], 'beta': []}
    PSD_2 = []
    for ELECTRODES in ['32electrodes', '32electrodes-old']:
        folder_path = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', ELECTRODES,  ACTIVE_SHAM, folder2)
        for patient in os.listdir(folder_path):
            file = glob.glob(os.path.join(folder_path, patient) + '/*' + 'Close.easy') if ELECTRODES == '32electrodes' else glob.glob(os.path.join(folder_path, patient) + '/*' + 'EEG.easy')
            for _ in file:
                raw_2 = data_transformation_easy(_)
                raw_2 = g1_preprocess(raw_2)
                psd_2, frequencies = mne.time_frequency.psd_array_welch(raw_2.get_data(), fmin=FMIN, fmax=FMAX, sfreq=SFREQ, n_fft = 8192)
                PSD_2.append(psd_2)

                if c==0:
                    duration = 4.0
                    overlap = 2.0 

                    samples_per_epoch = int(duration * SFREQ)
                    samples_per_overlap = int(overlap * SFREQ)

                    start, stop = 0, samples_per_epoch
                    events = []
                    while stop <= len(raw_2):
                        events.append([start, 0, 1]) 
                        start += samples_per_overlap
                        stop += samples_per_overlap

                    events = np.array(events)

                epochs_2 = mne.Epochs(raw_2, events, tmin=0, tmax=duration, baseline=(0,0), detrend=1,
                            picks=None, preload=True, reject=None)
                n_channels = len(raw_2.info['ch_names'])
                for i in mapping:
                    con = connectivity_matrix(epochs_2, i)
                    con_mat_2[mapping[i]].append(con.get_data().reshape((n_channels, n_channels)))
                c +=1 

    print(con_mat_2)

Creating RawArray with float64 data, n_channels=32, n_times=149999
    Range : 0 ... 149998 =      0.000 ...   299.996 secs
Ready.
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hann window with 0.0546 passband ripple and 44 dB stopband attenuation
- Lower transition bandwidth: 0.01 Hz
- Upper transition bandwidth: 11.25 Hz
- Filter length: 155001 samples (310.002 s)



/tmp/ipykernel_19086/607340406.py:2: RuntimeWarning: filter_length (155001) is longer than the signal (149999), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(method= 'fir',
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.7s


Effective window size : 16.384 (s)
Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computin

/tmp/ipykernel_19086/607340406.py:2: RuntimeWarning: filter_length (155001) is longer than the signal (149999), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(method= 'fir',
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.6s


Effective window size : 16.384 (s)
Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computin

...

Effective window size : 16.384 (s)
Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computin

/tmp/ipykernel_19086/607340406.py:2: RuntimeWarning: filter_length (155001) is longer than the signal (149999), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(method= 'fir',
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.0s


Effective window size : 16.384 (s)
Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computin

/tmp/ipykernel_19086/607340406.py:2: RuntimeWarning: filter_length (155001) is longer than the signal (74999), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(method= 'fir',
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Effective window size : 16.384 (s)
Not setting metadata
49 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computin

### PSD Plotting

PSD_x dimensions = no. of active/sham X n_channels X 23

In [6]:
labels = np.arange(0, FMAX)
avg_abs_power_1, avg_abs_power_2 = np.mean(np.mean(PSD_1, axis=0), axis=0), np.mean(np.mean(PSD_2, axis=0), axis=0) 

plt.figure()
plt.plot(frequencies, avg_abs_power_1, color='blue', label=f'{GROUP1}')
plt.plot(frequencies, avg_abs_power_2, color='red', label=f'{GROUP2}')
plt.title(f'Average Absolute Power in {ACTIVE_SHAM} treatment')
plt.xlabel('Frequency (Hz)')
plt.xticks(labels, labels, rotation ='vertical') 
plt.ylabel('PSD (μV²/Hz)')
plt.ylim([0, 2e-11])
plt.legend()

vertical_lines = list(set([limit for band in FREQ_BANDS.values() for limit in band]))
for line in vertical_lines:
    plt.axvline(x=line, color='gray', linestyle='--')
plt.show()

In [7]:
# Average con_mat_1 and con_mat_2
for i in mapping:
    connectivity[f'{GROUP1}-{mapping[i]}'] = np.mean(con_mat_1[mapping[i]], axis=0)
    connectivity[f'{GROUP2}-{mapping[i]}'] = np.mean(con_mat_2[mapping[i]], axis=0)

### PLOT FC for both pre and post

In [11]:
f, axarr = plt.subplots(1,3)
for count, i in enumerate(mapping):
    conmat1 = make_symmetric(connectivity[f'{GROUP1}-{mapping[i]}'], n_channels)

    im = axarr[count].imshow(conmat1)
    axarr[count].set_title(f'{mapping[i].title()}')
    axarr[count].set_xticks([_ for _ in range(n_channels)])
    axarr[count].set_xticklabels(raw_1.info['ch_names'], rotation='vertical')
    axarr[count].set_yticks([_ for _ in range(n_channels)])
    axarr[count].set_yticklabels(raw_1.info['ch_names'], rotation='horizontal')
    axarr[count].grid(True, which='both', linestyle='--', linewidth=0.5)

f.text(0.5, 0.04, 'X-axis label', ha='center')
f.text(0.04, 0.5, 'Y-axis label', va='center', rotation='vertical')
# f.colorbar(im, ax=axarr[count])
plt.show()

In [9]:
f, axarr = plt.subplots(1,3) 
for count, i in enumerate(mapping):
    conmat2 = make_symmetric(connectivity[f'{GROUP2}-{mapping[i]}'], n_channels)

    im = axarr[count].imshow(conmat2)
    axarr[count].set_title(f'{mapping[i].title()}')
    axarr[count].set_xticks([_ for _ in range(n_channels)])
    axarr[count].set_xticklabels(raw_1.info['ch_names'], rotation='vertical')
    axarr[count].set_yticks([_ for _ in range(n_channels)])
    axarr[count].set_yticklabels(raw_1.info['ch_names'], rotation='horizontal')
    axarr[count].grid(True, which='both', linestyle='--', linewidth=0.5)

f.text(0.5, 0.04, 'X-axis label', ha='center')
f.text(0.04, 0.5, 'Y-axis label', va='center', rotation='vertical')
plt.show()

### Thresholding Algorithm

In [ ]:
n_perms = 1000
n_network = n_perms//10
thresholds = np.linspace(0.1, 1.6, 200)
swn_z_1, swn_z_2 = {}, {}
optimal_threshold = {}
optimal_swn_perms, optimal_swn_real1, optimal_swn_real2 = {}, {}, {}
moving_avg1, moving_avg2 = {}, {}

for i in mapping:
    conmat1, conmat2 = connectivity[f'{GROUP1}-{mapping[i]}'], connectivity[f'{GROUP2}-{mapping[i]}']
    swn_z_1[mapping[i]], swn_z_2[mapping[i]] = np.zeros(len(thresholds)), np.zeros(len(thresholds))
    optimal_swn_perms[mapping[i]], optimal_swn_real1[mapping[i]], optimal_swn_real2[mapping[i]] = [[] for _ in range(len(thresholds))], np.zeros(len(thresholds)),  np.zeros(len(thresholds))
    max_diff = 0
    for threshi, threshold in enumerate(thresholds):
        swn_1, n_connections1 = calculate_swn(conmat1, threshold)
        swn_2, n_connections2 = calculate_swn(conmat2, threshold)

        # Create #n_network random matrices with vertices: n_channels and edges: average of edges of thresholded binary matrix of both groups 
        # And make a list of #n_network lists of clustering coeffs and shortest avg path length
        Crand, Lrand = [], []
        swn_perms = np.zeros(n_perms)
        for perm in range(n_network):
            random_binmat = create_random_graph(n_channels, (n_connections1+n_connections2)//2)
            random_binmat = make_symmetric(random_binmat, n_channels)
            grand = calculate_avergae_components(nx.from_numpy_array(random_binmat))
            Crand.append(grand[3])
            Lrand.append(grand[0])

        # choose any 2 random network from above n_networks #n_perms times 
        for perm in range(n_perms):
            whichnetworks2use = random.sample(range(n_network), 2)
            if Crand[whichnetworks2use[1]] and Lrand[whichnetworks2use[0]]:
                swn_perms[perm] = (Crand[whichnetworks2use[0]] / Crand[whichnetworks2use[1]]) / (Lrand[whichnetworks2use[0]] / Lrand[whichnetworks2use[1]])
            else:
                perm -= 1
    
        swn_rand_avg = np.mean(Crand, axis=0)/np.mean(Lrand, axis=0)
        swn_real_1 = swn_1 / swn_rand_avg
        swn_real_2 = swn_2 / swn_rand_avg
        val1, val2 = (swn_real_1 - np.mean(swn_perms)) / np.std(swn_perms), (swn_real_2 - np.mean(swn_perms)) / np.std(swn_perms)
        
        swn_z_1[mapping[i]][threshi] = val1
        swn_z_2[mapping[i]][threshi] = val2


        max_diff = 0
        moving_avg1[mapping[i]] = moving_average(swn_z_1[mapping[i]])
        moving_avg2[mapping[i]] = moving_average(swn_z_2[mapping[i]])
        moving_threshold = moving_average(thresholds)

        optimal_swn_perms[mapping[i]][threshi] = swn_perms
        optimal_swn_real1[mapping[i]][threshi] =swn_real_1
        optimal_swn_real2[mapping[i]][threshi] =swn_real_2

In [ ]:
for i in mapping:
    print(np.mean(optimal_swn_perms[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)]))

In [ ]:
# To maximize the statistical significance of small world networkness for both groups while 
# maximizing the difference between the z-normalized small world networks of the two groups.
f, axarr = plt.subplots(1,3, figsize=(100, 5))
index = {}
for count, i in enumerate(mapping):
    max_diff = 0
    for mov in range(len(moving_threshold)):
        x = np.abs(moving_avg1[mapping[i]][mov] - moving_avg2[mapping[i]][mov])
        if x>max_diff:
            max_diff = x
            optimal_threshold[mapping[i]] = moving_threshold[mov]
            index[mapping[i]] = mov

    # TODO: save fig file
    p_value_1 = np.mean(optimal_swn_perms[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)] >= optimal_swn_real1[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)])
    p_value_2 = np.mean(optimal_swn_perms[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)] >= optimal_swn_real2[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)])
    print(mapping[i], "p-value: ", p_value_1, p_value_2)

    axarr[count].hist(optimal_swn_perms[mapping[i]][index[mapping[i]]+(WINDOW_SIZE//2)], bins=50, color='b', alpha=0.7)
    axarr[count].axvline(optimal_swn_real1[mapping[i]][mov+(WINDOW_SIZE//2)], color='m', linewidth=3, label=f'{GROUP1}')
    axarr[count].axvline(optimal_swn_real2[mapping[i]][mov+(WINDOW_SIZE//2)], color='r', linewidth=3, label=f'{GROUP2}')
    axarr[count].set_xlabel(mapping[i])
    axarr[count].set_xlim([0.5,3])
    # axarr[count].set_ylim([0,100])
    axarr[count].legend()
f.suptitle('Random Network distribution for both Pre and Post Intervention')
plt.show()

In [ ]:
# PLOT SWN_Z vs THRESHOLDS
for count, i in enumerate(mapping):
    # High standardized network metric suggests that our network exhibits properties away from random networks 
    plt.plot(thresholds, swn_z_1[mapping[i]], '-o', markerfacecolor='w', label=f'{GROUP1}')
    plt.plot(thresholds, swn_z_2[mapping[i]], '-o', markerfacecolor='w', label=f'{GROUP2}')
    plt.xlabel(f'Threshold (Standard deviations above median) at frequency band {mapping[i]}')
    plt.ylabel(f'Normalized SWN')
    plt.xlim([0, 1.5])
    plt.legend()
    vline = ((index[mapping[i]]+WINDOW_SIZE//2)*1.5)/200
    plt.axvline(x=vline, color='gray', linestyle='--')
    plt.show()

### Binarize FC matrix

In [ ]:
for i in mapping:
    conmat1, conmat2 = connectivity[f'{GROUP1}-{mapping[i]}'], connectivity[f'{GROUP2}-{mapping[i]}']
    tthresh1 = np.median(conmat1[conmat1!=0]) + optimal_threshold[mapping[i]]*np.std(conmat1[conmat1!=0])
    tthresh2 = np.median(conmat2[conmat2!=0]) + optimal_threshold[mapping[i]]*np.std(conmat2[conmat2!=0])
    binarized_matrix[f'{GROUP1}-{mapping[i]}'] = connectivity[f'{GROUP1}-{mapping[i]}'] > tthresh1
    binarized_matrix[f'{GROUP2}-{mapping[i]}'] = connectivity[f'{GROUP2}-{mapping[i]}'] > tthresh2
    print(tthresh1, tthresh2)


### Find number of links in each band

In [ ]:
l1 = {}
l2 = {}
for i in mapping:
    links_g1 = 0
    links_g2 = 0

    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                links_g1+=1
            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                links_g2+=1

    print(f'{GROUP1}-{mapping[i]}-number of links: K=', links_g1)
    print(f'{GROUP2}-{mapping[i]}-number of links: K=', links_g2)

    l1[mapping[i]] = links_g1
    l2[mapping[i]] = links_g2

### Find Asymmetry in  each band

In [ ]:
left_region = ['Fp1', 'AF3', 'PO3','O1', 'CP1', 'FC1', 'P3', 'C3', 'F3', 'F7', 'FC5', 'CP5', 'T7', 'P7']
right_region = ['P8', 'T8', 'CP6', 'FC6', 'F8', 'F4', 'C4', 'P4', 'AF4', 'Fp2', 'FC2', 'CP2', 'O2', 'PO4']

a1 = {}
a2 = {}
for i in mapping:
    lr = 1
    rr = 1
    lrc = 1
    rrc = 1
    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lr += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rr += 1

            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lrc += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rrc += 1

    print(f"{GROUP1}: ",mapping[i], "left region: ", lr, "right region: ", rr, "proportion: ", lr/rr)
    print(f"{GROUP2}: ", mapping[i], "left region: ", lrc, "right region: ", rrc, "proportion: ", lrc/rrc)

    a1[mapping[i]] = lr/rr
    a2[mapping[i]] = lrc/rrc


### Calculate Difference matrix

In [ ]:
def return_index(channels_list, raw):
    indexes = []
    for channel in channels_list:
        for i in range(len(raw.info['ch_names'])):
            if raw.info['ch_names'][i]==channel:
                indexes.append(i)
    return(indexes)

In [ ]:
central_channels = return_index(['Pz', 'Oz', 'Fz', 'Cz'], raw_1)
central_channels
raw_1.info['bads'] = []

In [ ]:
numpy.set_printoptions(threshold=sys.maxsize)
for i in mapping:
    print((binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype('int')))

In [ ]:
difference_matrix = {}

f, axarr = plt.subplots(1, 3, figsize=(10,3))
for count, i in enumerate(mapping):
    difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) 
    for k in range(n_channels):
        for l in range(n_channels):
            if difference_matrix[mapping[i]][k][l] == -1 or k in central_channels or l in central_channels:
                difference_matrix[mapping[i]][k][l] = 0

    difference_matrix[mapping[i]] = make_symmetric(difference_matrix[mapping[i]], n_channels)

    diff_mat = difference_matrix[mapping[i]]
    axarr[count].imshow(diff_mat, interpolation='none')
    axarr[count].set_title(mapping[i])
    axarr[count].set_xticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='vertical')
    axarr[count].set_yticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='horizontal')
    axarr[count].grid(True, which='both', linestyle='--', linewidth=0.5)

   
    # plot_sensors_connectivity(
    #     raw_1.info,
    #     difference_matrix[mapping[i]],
    #     cbar_label=f'{mapping[i]}-Connectivity',
    #     )

f.suptitle(f'Binary FC difference in {ACTIVE_SHAM} treatment')
plt.show()

In [ ]:
def calculate_hub(mat):
        hub =[]
        for ch, ch_name in enumerate(raw_1.info['ch_names']):
            hub.append(np.count_nonzero(mat[ch] == 1))
        return hub

In [ ]:
edges_threshold = {}
for i in mapping:
    hub = calculate_hub(difference_matrix[mapping[i]])
    edges_threshold[mapping[i]] = np.median(hub) + np.std(hub)
    print(hub)
print(edges_threshold)

In [ ]:
selected_channels = {}
for i in mapping:
    selected_channels[mapping[i]] = []
    for ch in range(n_channels):
        if len(nx.from_numpy_array(difference_matrix[mapping[i]]).edges(ch)) > edges_threshold[mapping[i]]:
            selected_channels[mapping[i]].append(raw_1.info['ch_names'][ch])
print(selected_channels)

Create dictionary of Regions in brain vs channels

In [ ]:
regionvschannel = {
    'LC': ['C3', 'CP1', 'CP5', 'FC1', 'FC5'], # Left central 
    'LF': ['FP1', 'AF3', 'F3', 'F7'], # Left frontal
    'LT': ['T7'], # Left temporal
    'LPO': ['PO3',  'P3', 'P7', 'O1'], # Left parietal-occipital
    'RC': ['CP6', 'FC6', 'C4', 'FC2', 'CP2',], # Right central 
    'RF': ['F8', 'F4', 'AF4', 'FP2'], # Right frontal
    'RT': ['T8'], # Right temporal
    'RPO': ['P8', 'P4', 'PO4', 'O2'] , # Right parietal-occipital
}

In [ ]:
channel_to_region = {}
for region, region_channels in regionvschannel.items():
    for channel in region_channels:
        channel_to_region[channel] = region

selected_regions = {}
for band, channels in selected_channels.items():
    regions = set()
    for channel in channels:
        if channel in channel_to_region:
            regions.add(channel_to_region[channel])
    selected_regions[band] = list(regions)

print(selected_regions)